In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd '/content/gdrive/My Drive/Colab Notebooks/DL4H Project' 

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/DL4H Project


In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import spacy
# import re
# from tqdm import tqdm
# import string

In [ ]:
# # STEP 1: load Note datasets
# df_notes = pd.read_csv('NOTE_DATA_PATH/NOTEEVENTS.csv')

In [ ]:
# # STEP 2: Generate the cohort to pretrain on (here e.g. we only pretrian on physician and nursing notes) 
# df_notes = df_notes[df_notes.CATEGORY.isin(['Physician ','Nursing','Nursing/Others'])]

# # IMPORTANT: if you fine tune on the same dataset that you use for pretrain, you need to preclude the fine-tune test admissions 
# df_test_ids = pd.read_csv('FINETUNE_DATA_PATH/test.csv').HADM_ID.unique()
# df_notes_fold = df_notes[~df_notes.HADM_ID.isin(df_test_ids)]

In [ ]:
# df_notes_fold = pd.read_csv('./data/good_datasets/fold1/discharge/train.csv',nrows=13000)

In [ ]:
# # STEP 3: Preprocessing
# def preprocess1(x):
#     y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
#     y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
#     y=re.sub('dr\.','doctor',y)
#     y=re.sub('m\.d\.','md',y)
#     y=re.sub('admission date:','',y)
#     y=re.sub('discharge date:','',y)
#     y=re.sub('--|__|==','',y)
    
#     # remove, digits, spaces
#     y = y.translate(str.maketrans("", "", string.digits))
#     y = " ".join(y.split())
#     return y

# def preprocessing(df_notes): 
#     df_notes['TEXT']=df_notes['TEXT'].fillna(' ')
#     df_notes['TEXT']=df_notes['TEXT'].str.replace('\n',' ')
#     df_notes['TEXT']=df_notes['TEXT'].str.replace('\r',' ')
#     df_notes['TEXT']=df_notes['TEXT'].apply(str.strip)
#     df_notes['TEXT']=df_notes['TEXT'].str.lower()

#     df_notes['TEXT']=df_notes['TEXT'].apply(lambda x: preprocess1(x))
    
#     return df_notes

# df_notes_fold = preprocessing(df_notes_fold)

In [ ]:
# # STEP 4: Notes to Sentences
# from spacy.lang.en import English
# nlp = English()  # just the language with no model
# nlp.add_pipe(nlp.create_pipe('sentencizer'))

# # nlp praser may not work when there is only one token. In these cases, we just remove them as note that has length 1 usually is some random stuff

# def toSentence(x):
#     doc = nlp(x)
#     text=[]
#     try:
#         for sent in doc.sents:
#             st=str(sent).strip() 
#             if len(st)<20:
#                 #a lot of abbreviation is segmented as one line. But these are all describing the previous things
#                 #so I attached it to the sentence before
#                 if len(text)!=0:
#                     text[-1]=' '.join((text[-1],st))
#                 else:
#                     text=[st]
#             else:
#                 text.append((st))
#     except:
#         print(doc)
#     return text

# pretrain_sent=df_notes_fold['TEXT'].apply(lambda x: toSentence(x))

In [ ]:
# # STEP 5: Create Pretraining File
# file=open('PRETRAIN_DATA_PATH/clinical_sentences_pretrain_fold11.txt','w')
# pretrain_sent = pretrain_sent.values
# for i in tqdm(range(len(pretrain_sent))):
#     if len(pretrain_sent[i]) > 0:
#         # remove the one token note
#         note = pretrain_sent[i]
#         for sent in note:
#             file.write(sent+'\n')
#         file.write('\n')

In [ ]:
# STEP 6: Download the Implementations and Initial Checkpoint with the SentencePiece Model and Vocab 
# Github Repo:
# XLNet: git clone https://github.com/zihangdai/xlnet.git
# BERT: git clone https://github.com/google-research/bert.git
# Model: 
# XLNet-Base: https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
# BERT-Base-Uncased: https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

In [ ]:
#  %tensorflow_version 1.x
# !python create_pretraining_data.py \
#   --input_file=PRETRAIN_DATA_PATH/clinical_sentences_pretrain_fold11.txt \
#   --output_file=PRETRAIN_DATA_PATH/tf_examples_128_fold11.tfrecord \
#   --vocab_file=INITIAL_MODEL_PATH/vocab.txt \
#   --do_lower_case=True \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=3

In [ ]:
# !python create_pretraining_data.py \
#   --input_file=PRETRAIN_DATA_PATH/clinical_sentences_pretrain_fold11.txt \
#   --output_file=PRETRAIN_DATA_PATH/tf_examples_512_fold11.tfrecord \
#   --vocab_file=INITIAL_MODEL_PATH/vocab.txt \
#   --do_lower_case=True \
#   --max_seq_length=512 \
#   --max_predictions_per_seq=76 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=3


In [ ]:
#  %tensorflow_version 1.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [ ]:
#  %tensorflow_version 1.x
# !python ./run_pretraining.py \
# --input_file ./PRETRAIN_DATA_PATH/tf_examples_128_fold12.tfrecord \
# --output_dir ./PRETRAINED_MODEL_PATH/pretraining_output_discharge_fold12 \
# --do_train \
# --do_eval \
# --bert_config_file ./INITIAL_DATA_PATH/bert_config.json \
# --init_checkpoint ./PRETRAINED_MODEL_PATH/pretraining_output_discharge_fold12/model.ckpt-50000  \
# --train_batch_size 64 \
# --max_seq_length 128 \
# --max_predictions_per_seq 20 \
# --num_train_steps 99000 \
# --num_warmup_steps 10 \
# --learning_rate 2e-5 \
# # --tpu_name grpc://10.54.116.154:8470 \
# # --use_tpu \
#   #--master grpc://P10.61.58.114:8470
# # --init_checkpoint ./INITIAL_DATA_PATH/bert_model.ckpt \


In [ ]:
# %tensorflow_version 1.x
# !python ./run_pretraining.py \
#   --input_file ./PRETRAIN_DATA_PATH/tf_examples_512_fold12.tfrecord \
#   --output_dir ./PRETRAINED_MODEL_PATH/pretraining_output_discharge_fold12_512_v3\
#   --do_train \
#   --do_eval \
#   --bert_config_file ./INITIAL_DATA_PATH/bert_config.json \
#   --init_checkpoint ./PRETRAINED_MODEL_PATH/pretraining_output_discharge_fold12_512_v2/model.ckpt-13000 \
#   --train_batch_size 8 \
#   --max_seq_length 512 \
#   --max_predictions_per_seq 76 \
#   --num_train_steps 100000 \
#   --num_warmup_steps 10 \
#   --learning_rate 2e-5

In [ ]:
export BERT_BASE_DIR=./pytorch_discharge

transformers-cli convert --model_type bert \
  --tf_checkpoint $BERT_BASE_DIR/model.ckpt-7000 \
  --config $BERT_BASE_DIR/bert_config.json \
  --pytorch_dump_output $BERT_BASE_DIR/pytorch_model.bin

In [3]:
pip install funcsigs

In [4]:
pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 12.3 MB/s 
     |████████████████████████████████| 132 kB 54.2 MB/s 
     |████████████████████████████████| 79 kB 7.7 MB/s 
     |████████████████████████████████| 8.7 MB 51.1 MB/s 
     |████████████████████████████████| 138 kB 55.9 MB/s 
     |████████████████████████████████| 127 kB 51.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!python ./run_readmission.py \
  --task_name readmission \
  --readmission_mode discharge \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/discharge \
  --bert_model ./pytorch_discharge\
  --max_seq_length 512 \
  --train_batch_size 8\
  --output_dir ./result_early3_2
  ##same code just run again

Streaming output truncated to the last 5000 lines.
 86% 1324/1532 [01:52<00:17, 11.71it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
 87% 1326/1532 [01:53<00:17, 11.73it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:
!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode discharge \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/discharge \
  --bert_model ./pytorch_discharge\
  --max_seq_length 512 \
  --train_batch_size 8\
  --output_dir ./result_early5

Streaming output truncated to the last 5000 lines.
 86% 1324/1532 [01:53<00:17, 11.65it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
 87% 1326/1532 [01:53<00:17, 11.65it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:

!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode discharge \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/discharge \
  --bert_model bert-base-uncased\
  --max_seq_length 512 \
  --train_batch_size 8\
  --output_dir ./result_early4

Streaming output truncated to the last 5000 lines.
 86% 1324/1532 [01:53<00:17, 11.64it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
 87% 1326/1532 [01:53<00:17, 11.63it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:
!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode discharge \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/discharge \
  --bert_model ./pytorch_discharge\
  --max_seq_length 512 \
  --train_batch_size 8\
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --output_dir ./result_early7

!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode discharge \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/discharge \
  --bert_model ./model/bert_base\
  --max_seq_length 512 \
  --train_batch_size 8\
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --output_dir ./result_early8

Streaming output truncated to the last 5000 lines.
 86% 1324/1532 [01:53<00:17, 11.71it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
 87% 1326/1532 [01:53<00:17, 11.71it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:


!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode early \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/3days \
  --bert_model ./pytorch_early\
  --max_seq_length 512 \
  --train_batch_size 8\
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --output_dir ./result_early9


Streaming output truncated to the last 5000 lines.
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
 92% 2514/2721 [03:34<00:17, 11.70it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:
python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode discharge \
  --do_eval \
  --data_dir ./data/good_datasets/fold2/discharge/ \
  --bert_model ./model/discharge_readmission \
  --max_seq_length 512 \
  --output_dir ./result_discharge

In [ ]:

!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode early \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/3days \
  --bert_model ./model/bert_base\
  --max_seq_length 512 \
  --train_batch_size 8\
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --output_dir ./result_early10_v2

Streaming output truncated to the last 5000 lines.
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:


!python ./run_readmission_v2.py \
  --task_name readmission \
  --readmission_mode early \
  --do_train \
  --do_eval \
  --data_dir ./data/good_datasets/fold1/3days \
  --bert_model ./pytorch_early_v3\
  --max_seq_length 512 \
  --train_batch_size 8\
  --learning_rate 2e-5 \
  --num_train_epochs 3 \
  --output_dir ./result_early11


Streaming output truncated to the last 5000 lines.
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
 92% 2514/2721 [03:33<00:17, 11.78it/s]layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module
layer_module


In [ ]:
# # STEP 7: Generate Pretraining Tensorflow TF_Records

# # For Clinical BERT
# # cd to the git repo

# # Generate datasets for 128 max seq
# !python create_pretraining_data.py \
#   --input_file=PRETRAIN_DATA_PATH/clinical_sentences_pretrain_fold11.txt \
#   --output_file=PRETRAIN_DATA_PATH/tf_examples_128_fold11.tfrecord \
#   --vocab_file=INITIAL_MODEL_PATH/vocab.txt \
#   --do_lower_case=True \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=3

# # Generate datasets for 512 max seq
# !python create_pretraining_data.py \
#   --input_file=PRETRAIN_DATA_PATH/clinical_sentences_pretrain.txt \
#   --output_file=PRETRAIN_DATA_PATH/tf_examples_512.tfrecord \
#   --vocab_file=INITIAL_MODEL_PATH/vocab.txt \
#   --do_lower_case=True \
#   --max_seq_length=512 \
#   --max_predictions_per_seq=76 \
#   --masked_lm_prob=0.15 \
#   --random_seed=12345 \
#   --dupe_factor=3

# # For Clinical XLNet

# !python data_utils.py \
#     --bsz_per_host=6 \
#     --num_core_per_host=1 \
#     --seq_len=512 \
#     --reuse_len=256 \
#     --input_glob=/scratch/kh2383/MechVent/data/clinical_sentences_pretrain_xlnet.txt \
#     --save_dir=/scratch/kh2383/MechVent/data/xlnet_tfrecord/ \
#     --num_passes=5 \
#     --bi_data=True \
#     --sp_path=/scratch/kh2383/clibert/xlnet_cased_L-12_H-768_A-12/spiece.model \
#     --mask_alpha=6 \
#     --mask_beta=1 \
#     --num_predict=85

In [ ]:
# # STEP 8: Pretraining Use Original TF implementation

# # For Clinical BERT

# # First pretrain 100000 steps on the max seq length of 128
# !python run_pretraining.py \
#   --input_file=PRETRAIN_DATA_PATH/tf_examples_128.tfrecord \
#   --output_dir=PRETRAINED_MODEL_PATH/pretraining_output \
#   --do_train=True \
#   --do_eval=True \
#   --bert_config_file=INITIAL_DATA_PATH/bert_config.json \
#   --init_checkpoint=INITIAL_DATA_PATH/bert_model.ckpt \
#   --train_batch_size=64 \
#   --max_seq_length=128 \
#   --max_predictions_per_seq=20 \
#   --num_train_steps=100000 \
#   --num_warmup_steps=10 \
#   --learning_rate=2e-5

# # Then further pretrain 100000 steps on the max seq length of 512
# # NOTE: the init_checkpoint should switch to the 128 pretrained model

# !python run_pretraining.py \
#   --input_file=PRETRAIN_DATA_PATH/tf_examples_512.tfrecord \
#   --output_dir=PRETRAINED_MODEL_PATH/pretraining_output \
#   --do_train=True \
#   --do_eval=True \
#   --bert_config_file=INITIAL_DATA_PATH/bert_config.json \
#   --init_checkpoint=PRETRAINED_MODEL_PATH/pretraining_output_128/model.ckpt-100000 \
#   --train_batch_size=16 \
#   --max_seq_length=512 \
#   --max_predictions_per_seq=76 \
#   --num_train_steps=100000 \
#   --num_warmup_steps=10 \
#   --learning_rate=2e-5


# # For Clinical XLNet

# # Pretrain for 200000 steps

# !python train_gpu.py \
#     --record_info_dir=PRETRAIN_DATA_PATH/xlnet_tfrecord/tfrecords/ \
#     --model_dir=PRETRAINED_MODEL_PATH/xlnet_model/ \
#     --init_checkpoint=INITIAL_DATA_PATH/xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt \
#     --train_batch_size=8 \
#     --seq_len=512 \
#     --reuse_len=256 \
#     --mem_len=384 \
#     --perm_size=256 \
#     --n_layer=12 \
#     --d_model=768 \
#     --d_embed=768 \
#     --n_head=12 \
#     --d_head=64 \
#     --d_inner=3072 \
#     --untie_r=True \
#     --mask_alpha=6 \
#     --mask_beta=1 \
#     --num_predict=85 \
#     --num_hosts=1 \
#     --num_core_per_host=2 \
#     --train_steps=200000 \
#     --iterations=500 \
#     --save_steps=5000 